* heatstroke（救急搬送患者数）の表のスクレイピングができていない週があったため修正
* 元のコードを修正→こちらのコードでいったん記録済みjsonから抜いて回しなおした

In [1]:
#熱中症の救急搬送患者数（総務省）
#旧hot_weather
#（他のコードもすべて猛暑関連で後から認識しにくいので変更）
#ソース：消防庁
#毎週火曜（夕方？）に週次データをPDFで発表
#https://www.fdma.go.jp/disaster/heatstroke/post3.html

import pandas as pd
from datetime import datetime, date, timedelta
import tabula
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
#1) 週次のPDFファイルのURLを取得する
##ポータルの中身をみる
url = 'https://www.fdma.go.jp/disaster/heatstroke/post3.html'
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text, features="lxml")
##pdfのURLをいったん全部リストに入れる
pattern = re.compile(r'.*heatstroke_sokuhouti.*\.pdf')
href_list = [a['href'] for a in soup.select('div > a[href]') if pattern.match(a['href'])]

In [3]:
href_list

['/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230925.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230925.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230918.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230911.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230904.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230828.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230821.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230814.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230807-2.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230731.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230724.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230717.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230710.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230703.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sok

In [4]:
###毎回全部のURLをループするのは時間／安定性の面でよくないので
###過去分は蓄積しておき、新規分のみデータを取って積み上げる
#3) 前回までのデータ
filepath = './data/heatstroke.csv'
heatstroke = pd.read_csv(filepath, parse_dates=['date'])

In [5]:
heatstroke.head()

,date,熱中症患者数
0,2023-05-01,46.0
1,2023-05-02,38.0
2,2023-05-03,94.0
3,2023-05-04,143.0
4,2023-05-05,117.0


In [6]:
#日付ごと欠損
set(pd.date_range(heatstroke.date.min(), heatstroke.date.max())) - set(heatstroke.date)

set()

In [7]:
#日付のみでデータのない日
null_dates = heatstroke[heatstroke['熱中症患者数'].isna()]

In [8]:
null_dates = null_dates.date.dt.strftime('%Y%m%d').to_list()

In [9]:
null_dates

['20230821',
 '20230822',
 '20230823',
 '20230824',
 '20230825',
 '20230826',
 '20230827',
 '20230904',
 '20230905',
 '20230906',
 '20230907',
 '20230908',
 '20230909',
 '20230910',
 '20230911',
 '20230912',
 '20230913',
 '20230914',
 '20230915',
 '20230916',
 '20230917',
 '20230918',
 '20230919',
 '20230920',
 '20230921',
 '20230922',
 '20230923',
 '20230924']

In [10]:
#データ取得ができなかったURL
href_null = [h for h in href_list if any(null_date in h for null_date in null_dates)]

In [11]:
href_null

['/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230918.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230911.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230904.pdf',
 '/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230821.pdf']

jsonを書き換え

In [12]:
#前回までに取得済みと記録されているURL
with open('./data/heatstroke-pdf-list.json', 'r') as file:
    href_list_prev  = json.load(file)

In [13]:
href_list = list(set(h for h in href_list_prev if h not in href_null))

In [14]:
with open('./data/heatstroke-pdf-list.json', 'w') as f:
    json.dump(href_list, f)